# Indexação de Vetores no Qdrant
Neste notebook, realizamos a indexação dos vetores (embeddings) dos itens no banco de dados vetorial **Qdrant**.
Utilizamos a "Torre de Itens" do nosso modelo Two-Tower treinado para gerar os vetores de todos os itens do catálogo.
Isso permitirá realizar buscas por similaridade (Retrieval) de forma eficiente em tempo real.

In [1]:
import torch
import polars as pl
import pandas as pd
from qdrant_client import QdrantClient
from qdrant_client.http import models
from pathlib import Path
from tqdm import tqdm
import json
import numpy as np
import sys

sys.path.append('./src')

from model import TwoTowerModel

# Configs
MODEL_PATH = "./data/two_tower_model.ckpt"
METADATA_PATH = "./data/model_metadata.json"
ITEM_FEATURES_PATH = "./feature_repo/data/item_features.parquet"
QDRANT_HOST = "qdrant" # Nome do container no docker network
COLLECTION_NAME = "hm_items"

print("🚀 Iniciando Indexação no Qdrant...")

🚀 Iniciando Indexação no Qdrant...


## 1. Carregamento do Modelo Treinado
Carregamos o modelo Two-Tower salvo anteriormente (`.ckpt`).
Colocamos o modelo em modo de avaliação (`eval()`) e congelamos os gradientes, pois usaremos apenas para inferência (geração de embeddings).

In [2]:
# 1. Carregar Modelo
print("🧠 Carregando modelo treinado...")
model = TwoTowerModel.load_from_checkpoint(MODEL_PATH)
model.eval() # Modo de inferência
model.freeze() # Desativar gradientes

🧠 Carregando modelo treinado...


## 2. Preparação do Catálogo de Itens
Carregamos os dados dos itens que serão indexados.
Precisamos garantir que temos as mesmas features usadas no treinamento (`popularity_score`, `avg_price`) e os IDs mapeados corretamente (`item_index`).

In [4]:
# 2. Carregar Dados dos Itens (Para ter as features de input)
# Precisamos do item_index que criamos no passo 04. 
# Como o passo 04 salvou o dataset de treino com os indices, vamos pegar de lá os unicos.
print("📦 Carregando catálogo de itens...")
df_train = pd.read_parquet("./data/training_dataset.parquet", columns=["item_index", "article_id", "popularity_score", "avg_price"])
# Remover duplicatas para ter apenas itens únicos
unique_items = df_train.drop_duplicates(subset=["item_index"]).sort_values("item_index")

print(f"Total de itens únicos a indexar: {len(unique_items)}")

📦 Carregando catálogo de itens...
Total de itens únicos a indexar: 8451


## 3. Configuração do Qdrant
Conectamos ao serviço do Qdrant e recriamos a coleção.
Definimos a métrica de distância como **Cosseno**, que é compatível com a normalização L2 que aplicamos na saída do modelo.

In [6]:
# 3. Conectar ao Qdrant
client = QdrantClient(host=QDRANT_HOST, port=6333)

# Recriar coleção (apaga anterior se existir)
client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(size=32, distance=models.Distance.COSINE),
)

/tmp/ipykernel_3220/2188904423.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

## 4. Geração de Embeddings e Indexação
Iteramos sobre todos os itens do catálogo em batches.
Para cada batch:
1.  Passamos os dados pela **Item Tower** do modelo para gerar os vetores.
2.  Normalizamos os vetores.
3.  Enviamos os vetores e metadados (payload) para o Qdrant.

In [ ]:
# 4. Gerar Embeddings e Indexar em Batches
batch_size = 256
total_batches = len(unique_items) // batch_size + 1

print("⚡ Gerando vetores e enviando...")

# Processar em batches
for i in tqdm(range(0, len(unique_items), batch_size)):
    batch = unique_items.iloc[i : i + batch_size]
    
    # Preparar inputs para o PyTorch
    item_ids_tensor = torch.LongTensor(batch['item_index'].values)
    
    # Normalizar features numéricas (mesma lógica do treino)
    item_feats_tensor = torch.FloatTensor(batch[['popularity_score', 'avg_price']].values)
    
    # Passar pela ITEM TOWER
    with torch.no_grad():
        # Acessar apenas a sub-rede de itens
        i_emb = model.item_embedding(item_ids_tensor)
        i_input = torch.cat([i_emb, item_feats_tensor], dim=1)
        item_vectors = model.item_mlp(i_input)
        # Normalizar L2
        item_vectors = torch.nn.functional.normalize(item_vectors, p=2, dim=1)
    
    # Converter para lista Python
    vectors_list = item_vectors.numpy().tolist()
    
    # Preparar Payloads (Metadados para filtrar depois: Preço, Categoria, ID original)
    payloads = json.loads(batch.to_json(orient="records"))
    
    # IDs no Qdrant (usaremos o item_index como ID numérico do ponto)
    ids = batch['item_index'].tolist()
    
    # Upload
    client.upsert(
        collection_name=COLLECTION_NAME,
        points=models.Batch(
            ids=ids,
            vectors=vectors_list,
            payloads=payloads
        )
    )

print(f"✅ Indexação concluída! Coleção '{COLLECTION_NAME}' pronta com {client.count(COLLECTION_NAME).count} vetores.")

⚡ Gerando vetores e enviando...


100%|██████████| 34/34 [00:01<00:00, 18.33it/s]


✅ Indexação concluída! Coleção 'hm_items' pronta com 8451 vetores.
